# Data Harvesting Pipeline

This notebook runs the dataset pipeline `build_dataset_D(nqubits, topology; ...)` for:
- Ring graphs (cycle) for selected sizes
- Rectangle grids (rows × cols) for selected sizes
- IBM chips
- Barbel graphs

## 0) Setup

Install dependencies once (in a Julia REPL):
```julia
using Pkg
Pkg.add(["PauliPropagation","Graphs","DataFrames","CSV","JSON3", "Plots", "StatsPlots", "LaTeXStrings", "ProgressMeter"]) 
```


In [1]:
using PauliPropagation
using Graphs
using DataFrames
using CSV
using JSON3
using Plots
using Printf
using StatsPlots
using LaTeXStrings
using ProgressMeter

Configure the number of threads:

```PowerShell
# Windows (PowerShell)
[System.Environment]::SetEnvironmentVariable('JULIA_NUM_THREADS', '8', 'User')


/!\ Running this notebook on a 16GB RAM laptop showed memory was full for ring size 15 and rectangle size $9 \times 2$ harvesting.<br>
To avoid performance hits due to memory compression, we advise to keep the num_threads of two in order to have sufficient RAM-per-thread availability.<br>
Also note that this could be overcome by running this on a high-memory CPU cluster with large number of threads.

In [ ]:
using Base.Threads
println("Running with $(Threads.nthreads()) threads.")

Running with 8 threads.


## 1) Imports

In [ ]:
include(joinpath(@__DIR__, "src", "pauli_encoding.jl"));
include(joinpath(@__DIR__, "src", "symmetry_orbits.jl"));
include(joinpath(@__DIR__, "src", "trotter_model.jl"));
include(joinpath(@__DIR__, "src", "generators_P.jl"));
include(joinpath(@__DIR__, "src", "harvest_S.jl"));
include(joinpath(@__DIR__, "src", "targets_L2.jl"));
include(joinpath(@__DIR__, "src", "io_dataset.jl"));
include(joinpath(@__DIR__, "src", "run_tracker.jl"));
include(joinpath(@__DIR__, "src", "build_dataset.jl"));

## 2) Topology

In [ ]:
function ring_topology(n::Int)
    @assert n ≥ 3
    edges = Tuple{Int,Int}[]
    for i in 1:(n-1)
        push!(edges, (i, i+1))
    end
    push!(edges, (n, 1))
    return edges
end;

function rectangle_topology(rows::Int, cols::Int)
    @assert rows ≥ 1 && cols ≥ 1
    # map (r,c) -> linear index in 1..rows*cols
    idx(r,c) = (r-1)*cols + c
    edges = Tuple{Int,Int}[]
    for r in 1:rows
        for c in 1:cols
            if r < rows
                push!(edges, (idx(r,c), idx(r+1,c)))
            end
            if c < cols
                push!(edges, (idx(r,c), idx(r,c+1)))
            end
        end
    end
    return edges
end;

In [ ]:
rect_sizes = [(9,2), (12,2), (15,2), (18,2)];
ring_sizes = [15, 18];
# ring_sizes = [6, 9, 12, 15, 18];
# rect_sizes = [(6,2), (9,2), (12,2), (15,2), (18,2)];

### Optional topologies: 

In [ ]:
johannesburg_nqubits = 20;
johannesburg_topology = 
[
    (1,2), (2,3), (3,4), (4,5),
    (6,7), (7,8), (8,9), (9,10),
    (11,12), (12,13), (13,14), (14,15),
    (16,17), (17,18), (18,19), (19,20),
    (1,6), (5,10), (6,11), (10,15), (11,16), (15,20),
    (8,13)
];

In [ ]:
singapore_nqubits = 20;
singapore_topology = 
[
    (1,2), (2,3), (3,4), (4,5),
    (6,7), (7,8), (8,9), (9,10),
    (11,12), (12,13), (13,14), (14,15),
    (16,17), (17,18), (18,19), (19,20),
    (2,7), (4,9), (6,11), (8,13), (10,15), (12,17), (14,19)
];

In [ ]:
tokyo_nqubits = 20;
tokyo_topology = 
[
    (1,2), (1,6),
    (2,3), (2,7), (2,8),
    (3,7),
    (4,9),
    (5,9), (5,10),
    (6,7), (6,11), (6,12),
    (7,8), (7,11), (7,12),
    (8,9), (8,13), 
    (9,10), (9,13), (9,14),
    (11,12), (11,16),
    (12,13), (12,17), (12,18),
    (13,14), (13,17),
    (14,15), (14,19), (14,20),
    (15,19), (15,20), 
    (16,17), (17,18), (18,19)
];

In [ ]:
function barbell_topology(n_qubits::Int)
    @assert n ≥ 6
    n_bell = div(n_qubits - (isodd(n_qubits) ? 1 : 0), 2)
    n_bridge = n_qubits - 2 * n_bell
    return barbell_topology(n_bell, n_bridge)
end;

Sources: [[1]](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9504821), [[2]](https://arxiv.org/abs/2006.05205)

## 3) Batch runs

In [ ]:
# --- initial parameters ---

Base.@kwdef struct DatasetConfig
    # --- physics / trotter ---
    L1::Int = 32
    L2::Int = 2
    c1::Float64 = 1e-3
    c2::Float64 = 0.0
    T::Float64  = 1.0
    J::Float64  = 2.0
    h::Float64  = 1.0

    # --- generators ---
    include_single_Z::Bool = true
    include_double_Z::Bool = true

    # --- step 2 control ---
    max_reps::Int = 100_000
    reps_seed::Union{Nothing,Int} = 1234

    # --- instrumentation ---
    collect_stats_step1::Bool = true
    show_progress::Bool = true
    progress_every::Int = 5000
end

cfg = DatasetConfig()

DatasetConfig(32, 2, 0.001, 0.0, 1.0, 2.0, 1.0, true, true, 100000, 1234, true, true, 5000)

In [ ]:
function run(tag::String, n::Int, topo, cfg::DatasetConfig)
    outdir = joinpath(@__DIR__, "datasets", tag)
    mkpath(outdir)

    step1_plot = joinpath(outdir, "timing_step1.png")

    t0 = time_ns()
    results = build_dataset_D(
        n,
        topo;
        include_single_Z = cfg.include_single_Z,
        include_double_Z = cfg.include_double_Z,
        L1 = cfg.L1,
        L2 = cfg.L2,
        c1 = cfg.c1,
        c2 = cfg.c2,
        T  = cfg.T,
        J  = cfg.J,
        h  = cfg.h,

        save_step1 = joinpath(outdir, "S.csv"),
        save_rep   = joinpath(outdir, "rep.csv"),
        save_full  = joinpath(outdir, "full.csv"),

        collect_stats_step1 = cfg.collect_stats_step1,
        plot_step1_times    = step1_plot,
        show_progress       = cfg.show_progress,
        progress_every      = cfg.progress_every,

        max_reps  = cfg.max_reps,
        reps_seed = cfg.reps_seed,
    )
    t1 = time_ns()

    println("Saved step-1 timing plot: ", step1_plot)
    @printf("Total wall time for %s: %.3f s\n", tag, (t1 - t0) / 1e9)

    return results
end

results = Dict{String,Any}()

for (r, c) in rect_sizes
    n = r * c
    tag = "rect_" * string(r) * "x" * string(c)
    println("Running ", tag)
    results[tag] = run(tag, n, rectangle_topology(r, c), cfg)
    println()
end

for n in ring_sizes
    tag = "ring_" * string(n)
    println("Running ", tag)
    results[tag] = run(tag, n, ring_topology(n), cfg)
    println()
end

results

# Total wall time for rect_6x2: 45917.542 s

Running rect_6x2
Using Pauli integer type: PauliPropagation.UInt24
Step 1: harvesting observables (L1=32, c1=0.001)
Capping Step-2 reps: using 100000 / 100000 (seed=1234)
Step 2: computing features (L2=2, c2=0.0) on 100000 reps
Step 2 chunk 4: 5000/12500
Step 2 chunk 5: 5000/12500
Step 2 chunk 7: 5000/12500
Step 2 chunk 2: 5000/12500
Step 2 chunk 6: 5000/12500
Step 2 chunk 1: 5000/12500
Step 2 chunk 8: 5000/12500
Step 2 chunk 3: 5000/12500
Step 2 chunk 5: 10000/12500
Step 2 chunk 4: 10000/12500
Step 2 chunk 7: 10000/12500
Step 2 chunk 2: 10000/12500
Step 2 chunk 6: 10000/12500
Step 2 chunk 3: 10000/12500
Step 2 chunk 8: 10000/12500
Step 2 chunk 1: 10000/12500
Step 2 chunk files written: c:\Users\hugoi\OneDrive\Documents\Project2\MLforPauliProp\datasets\rect_6x2\rep_part_01.csv, c:\Users\hugoi\OneDrive\Documents\Project2\MLforPauliProp\datasets\rect_6x2\rep_part_02.csv, c:\Users\hugoi\OneDrive\Documents\Project2\MLforPauliProp\datasets\rect_6x2\rep_part_03.csv, c:\Users\hugoi\OneDrive\D